In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mygene
import h5py
import networkx as nx
import seaborn as sns
%matplotlib inline


## Organizando Labels


### Positivos

In [6]:
# 1ª lista na todas as outras vão ser baseadas é a lista de labels final do Shulte-Sasse, porque ela contém o número exato de genes que eu tenho disponíveis com features correspondentes.

label_unity = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_labels.tsv', sep= '\t')

label_unity

,gene,label
0,A1BG,False
1,A1CF,True
2,A2M,False
3,A2ML1,False
4,A3GALT2,False
...,...,...
19597,ZYG11A,False
19598,ZYG11B,False
19599,ZYX,False
19600,ZZEF1,False


In [77]:
label_unity['label'].value_counts()

False    18763
True       839
Name: label, dtype: int64

In [7]:
# Agora a partir das listas baixadas eu crio uma lista de labels própria (seguindo a quantidade de genes dentro das redes 19602) 
# Lista de positivos canonicos:
    # Canonicos do NCG e os Tier 1 CGC

lista_NCG = pd.read_csv('C:/Users/renan/Desktop/Labels/NCG_cancerdrivers_annotation_supporting_evidence.tsv', sep='\t')

lista_NCG

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",noncoding,NaN,NaN,NaN,NaN,NaN
8882,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",coding,NaN,NaN,NaN,NaN,NaN
8883,100423062,IGLL5,32024818,WGS,Multiple,multiple,pan-cancer_adult,DriverPower,noncoding,NaN,NaN,NaN,NaN,NaN
8884,100505993,TMEM210,27311963,WGS,Multiple,multiple,pan-cancer_adult,OncodriveFML,noncoding,NaN,NaN,NaN,NaN,NaN


In [8]:
lista_NCG['type'].value_counts()

WES                                    3254
WGS                                    1662
WGS-WES                                1190
Pan-cancer                             1050
Canonical Cancer Driver                 809
Gene Panel                              357
WES-Gene Panel                          346
WGS-WES-Gene Panel                      133
WES plus targeted deep resequencing      54
WGS-Gene Panel                           31
Name: type, dtype: int64

In [9]:
lista_NCG2 = lista_NCG[lista_NCG['type'] == 'Canonical Cancer Driver']
 
lista_NCG2

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
5,27,ABL2,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",NaN,NaN,1.0,0.0
17,90,ACVR1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,oncogene,NaN,NaN,1.0,0.0
32,91,ACVR1B,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,NaN,TSG,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752,345930,ECT2L,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,NaN,NaN,NaN,0.0,0.0
8757,346389,MACC1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,oncogene,NaN,NaN,1.0,0.0
8765,353088,ZNF429,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,NaN,NaN,NaN,0.0,0.0
8840,442444,FAM47C,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,NaN,NaN,NaN,0.0,0.0


In [10]:
lista_NCG3 = lista_NCG2.drop(columns=['entrez', 'pubmed_id', 'organ_system', 'primary_site', 'cancer_type', 'method', 'coding_status', 'cgc_annotation', 'vogelstein_annotation', 'saito_annotation', 'NCG_oncogene', 'NCG_tsg'])

lista_NCG3

,symbol,type
2,ABL1,Canonical Cancer Driver
4,ABL1,Canonical Cancer Driver
5,ABL2,Canonical Cancer Driver
17,ACVR1,Canonical Cancer Driver
32,ACVR1B,Canonical Cancer Driver
...,...,...
8752,ECT2L,Canonical Cancer Driver
8757,MACC1,Canonical Cancer Driver
8765,ZNF429,Canonical Cancer Driver
8840,FAM47C,Canonical Cancer Driver


In [18]:
lista_pos1 = lista_NCG3.drop_duplicates(subset = ['symbol'], keep = 'first')

lista_pos1

,symbol,type
2,ABL1,Canonical Cancer Driver
5,ABL2,Canonical Cancer Driver
17,ACVR1,Canonical Cancer Driver
32,ACVR1B,Canonical Cancer Driver
41,ACVR2A,Canonical Cancer Driver
...,...,...
8752,ECT2L,Canonical Cancer Driver
8757,MACC1,Canonical Cancer Driver
8765,ZNF429,Canonical Cancer Driver
8840,FAM47C,Canonical Cancer Driver


In [12]:
lista_NCG["labels"] = lista_NCG["NCG_oncogene"] + lista_NCG["NCG_tsg"]


lista_NCG

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg,labels
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN,NaN
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,1.0
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,1.0
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,1.0
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",noncoding,NaN,NaN,NaN,NaN,NaN,NaN
8882,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",coding,NaN,NaN,NaN,NaN,NaN,NaN
8883,100423062,IGLL5,32024818,WGS,Multiple,multiple,pan-cancer_adult,DriverPower,noncoding,NaN,NaN,NaN,NaN,NaN,NaN
8884,100505993,TMEM210,27311963,WGS,Multiple,multiple,pan-cancer_adult,OncodriveFML,noncoding,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
lista_NCG['labels'].value_counts()

1.0    4582
0.0     348
Name: labels, dtype: int64

In [14]:
lista_NCG5 = lista_NCG.drop(columns=['entrez', 'type', 'pubmed_id', 'organ_system', 'primary_site', 'cancer_type', 'method', 'coding_status', 'cgc_annotation', 'vogelstein_annotation', 'saito_annotation', 'NCG_oncogene', 'NCG_tsg'])

lista_NCG5

,symbol,labels
0,ABCF1,NaN
1,ABL1,1.0
2,ABL1,1.0
3,ABL1,1.0
4,ABL1,1.0
...,...,...
8881,IGLL5,NaN
8882,IGLL5,NaN
8883,IGLL5,NaN
8884,TMEM210,NaN


In [15]:
lista_NCG6 = lista_NCG5.dropna()

In [16]:
lista_NCG6.drop_duplicates(subset = ['symbol'], keep = 'first')

,symbol,labels
1,ABL1,1.0
5,ABL2,1.0
14,ACVR1,1.0
21,ACVR1B,1.0
34,ACVR2A,1.0
...,...,...
8752,ECT2L,0.0
8757,MACC1,1.0
8765,ZNF429,0.0
8840,FAM47C,0.0


In [19]:
# CGC

lista_cgc = pd.read_csv('C:/Users/renan/Desktop/Labels/Cancer_Gene_Census.csv')

lista_cgc

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,12.00,yes,NaN,GBM,NaN,NaN,O,NaN,NaN,Mis,NaN,NaN,NaN,"353088,ENSG00000197013.9,Q86V71,ZNF429"
725,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,11.20,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",NaN,NaN,E,NaN,NaN,Mis,NaN,NaN,NaN,"90827,ENSG00000185177.12,KR19,Q96JC4,ZNF479"
726,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,ALL,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"25925,EHZF,ENSG00000198795.10,Evi3,Q96K83,ZNF521"
727,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"84133,BK747E2.3,ENSG00000183579.15,FLJ22057,KI..."


In [20]:
lista_cgc2 = lista_cgc[['Gene Symbol','Tier']]

lista_cgc2

,Gene Symbol,Tier
0,A1CF,2
1,ABI1,1
2,ABL1,1
3,ABL2,1
4,ACKR3,1
...,...,...
724,ZNF429,2
725,ZNF479,2
726,ZNF521,1
727,ZNRF3,2


In [21]:
lista_cgc3 = lista_cgc2[lista_cgc2['Tier'] == 1]
 
lista_cgc3

,Gene Symbol,Tier
1,ABI1,1
2,ABL1,1
3,ABL2,1
4,ACKR3,1
5,ACSL3,1
...,...,...
720,ZMYM2,1
722,ZNF331,1
723,ZNF384,1
726,ZNF521,1


In [31]:
# Criando uma lista unificada com os canonicos e identificando a fonte

part1 = lista_NCG3.drop_duplicates(subset = ['symbol'], keep = 'first')

part1

,symbol,type
2,ABL1,Canonical Cancer Driver
5,ABL2,Canonical Cancer Driver
17,ACVR1,Canonical Cancer Driver
32,ACVR1B,Canonical Cancer Driver
41,ACVR2A,Canonical Cancer Driver
...,...,...
8752,ECT2L,Canonical Cancer Driver
8757,MACC1,Canonical Cancer Driver
8765,ZNF429,Canonical Cancer Driver
8840,FAM47C,Canonical Cancer Driver


In [32]:
part2 = lista_cgc3.rename(columns={"Gene Symbol": "symbol"})

part2

,symbol,Tier
1,ABI1,1
2,ABL1,1
3,ABL2,1
4,ACKR3,1
5,ACSL3,1
...,...,...
720,ZMYM2,1
722,ZNF331,1
723,ZNF384,1
726,ZNF521,1


In [35]:
part3 = oncokb_canon.rename(columns={"Hugo Symbol": "symbol"})

part3


,symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
0,ABL1,Yes,No,YesNo
1,AKT1,Yes,No,YesNo
2,ALK,Yes,No,YesNo
3,AMER1,No,Yes,NoYes
4,APC,No,Yes,NoYes
...,...,...,...,...
1052,WWP1,Yes,No,YesNo
1057,YY1,Yes,No,YesNo
1059,ZBTB20,Yes,Yes,YesYes
1061,ZFP36L1,No,Yes,NoYes


In [36]:
canonicos_list = pd.concat([part1, part2, part3])

canonicos_list

,symbol,type,Tier,Is Oncogene,Is Tumor Suppressor Gene,labels
2,ABL1,Canonical Cancer Driver,NaN,NaN,NaN,NaN
5,ABL2,Canonical Cancer Driver,NaN,NaN,NaN,NaN
17,ACVR1,Canonical Cancer Driver,NaN,NaN,NaN,NaN
32,ACVR1B,Canonical Cancer Driver,NaN,NaN,NaN,NaN
41,ACVR2A,Canonical Cancer Driver,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1052,WWP1,NaN,NaN,Yes,No,YesNo
1057,YY1,NaN,NaN,Yes,No,YesNo
1059,ZBTB20,NaN,NaN,Yes,Yes,YesYes
1061,ZFP36L1,NaN,NaN,No,Yes,NoYes


In [38]:
canonicos_list2 = canonicos_list.drop_duplicates(subset = ['symbol'], keep = 'first')

canonicos_list2

,symbol,type,Tier,Is Oncogene,Is Tumor Suppressor Gene,labels
2,ABL1,Canonical Cancer Driver,NaN,NaN,NaN,NaN
5,ABL2,Canonical Cancer Driver,NaN,NaN,NaN,NaN
17,ACVR1,Canonical Cancer Driver,NaN,NaN,NaN,NaN
32,ACVR1B,Canonical Cancer Driver,NaN,NaN,NaN,NaN
41,ACVR2A,Canonical Cancer Driver,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1052,WWP1,NaN,NaN,Yes,No,YesNo
1057,YY1,NaN,NaN,Yes,No,YesNo
1059,ZBTB20,NaN,NaN,Yes,Yes,YesYes
1061,ZFP36L1,NaN,NaN,No,Yes,NoYes


In [39]:
canonicos_list3 = canonicos_list2[['symbol']]

canonicos_list3

,symbol
2,ABL1
5,ABL2
17,ACVR1
32,ACVR1B
41,ACVR2A
...,...
1052,WWP1
1057,YY1
1059,ZBTB20
1061,ZFP36L1


### Candidatos

In [40]:
lista_NCG_candidatos = pd.read_csv('C:/Users/renan/Desktop/Labels/NCG_cancerdrivers_annotation_supporting_evidence.tsv', sep='\t')

lista_NCG_candidatos

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",noncoding,NaN,NaN,NaN,NaN,NaN
8882,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",coding,NaN,NaN,NaN,NaN,NaN
8883,100423062,IGLL5,32024818,WGS,Multiple,multiple,pan-cancer_adult,DriverPower,noncoding,NaN,NaN,NaN,NaN,NaN
8884,100505993,TMEM210,27311963,WGS,Multiple,multiple,pan-cancer_adult,OncodriveFML,noncoding,NaN,NaN,NaN,NaN,NaN


In [41]:
lista_NCG_candidatos2 = lista_NCG_candidatos[lista_NCG['type'] != 'Canonical Cancer Driver']
 
lista_NCG_candidatos2

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0
6,48,ACO1,24390350,Pan-cancer,Multiple,multiple,pan-cancer_adult,MutSig,coding,NaN,NaN,NaN,NaN,NaN
7,60,ACTB,30606723,WGS,Hematologic and lymphatic,blood,"dlblc, follicular_lymphoma",Paper-specific,noncoding,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",noncoding,NaN,NaN,NaN,NaN,NaN
8882,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",coding,NaN,NaN,NaN,NaN,NaN
8883,100423062,IGLL5,32024818,WGS,Multiple,multiple,pan-cancer_adult,DriverPower,noncoding,NaN,NaN,NaN,NaN,NaN
8884,100505993,TMEM210,27311963,WGS,Multiple,multiple,pan-cancer_adult,OncodriveFML,noncoding,NaN,NaN,NaN,NaN,NaN


In [42]:
lista_NCG_candidatos3 = lista_NCG_candidatos2[['symbol','type', 'cancer_type']]

lista_NCG_candidatos3

,symbol,type,cancer_type
0,ABCF1,WGS-WES,multiple_myeloma
1,ABL1,Pan-cancer,pan-cancer_adult
3,ABL1,Pan-cancer,pan-cancer_paediatric
6,ACO1,Pan-cancer,pan-cancer_adult
7,ACTB,WGS,"dlblc, follicular_lymphoma"
...,...,...,...
8881,IGLL5,WGS,burkitt lymphoma
8882,IGLL5,WGS,burkitt lymphoma
8883,IGLL5,WGS,pan-cancer_adult
8884,TMEM210,WGS,pan-cancer_adult


In [45]:
lista_NCG_candidatos4 = lista_NCG_candidatos3.drop_duplicates(subset = ['symbol'], keep = 'first')

lista_NCG_candidatos5 = lista_NCG_candidatos4[['symbol']]

lista_NCG_candidatos5

,symbol
0,ABCF1
1,ABL1
6,ACO1
7,ACTB
12,ACTG1
...,...
8870,MEF2B
8875,ZNF605
8876,IGLL5
8884,TMEM210


In [46]:
lista_cgc_candidatos = pd.read_csv('C:/Users/renan/Desktop/Labels/Cancer_Gene_Census.csv')

lista_cgc_candidatos

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,12.00,yes,NaN,GBM,NaN,NaN,O,NaN,NaN,Mis,NaN,NaN,NaN,"353088,ENSG00000197013.9,Q86V71,ZNF429"
725,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,11.20,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",NaN,NaN,E,NaN,NaN,Mis,NaN,NaN,NaN,"90827,ENSG00000185177.12,KR19,Q96JC4,ZNF479"
726,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,ALL,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"25925,EHZF,ENSG00000198795.10,Evi3,Q96K83,ZNF521"
727,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"84133,BK747E2.3,ENSG00000183579.15,FLJ22057,KI..."


In [47]:
lista_cgc_candidatos2 = lista_cgc_candidatos[['Gene Symbol','Tier']]

lista_cgc_candidatos2

,Gene Symbol,Tier
0,A1CF,2
1,ABI1,1
2,ABL1,1
3,ABL2,1
4,ACKR3,1
...,...,...
724,ZNF429,2
725,ZNF479,2
726,ZNF521,1
727,ZNRF3,2


In [48]:
lista_cgc_candidatos3 = lista_cgc_candidatos2[lista_cgc_candidatos2['Tier'] == 2]
 
lista_cgc_candidatos3

,Gene Symbol,Tier
0,A1CF,2
6,ACSL6,2
13,AKAP9,2
16,AKT3,2
17,ALDH2,2
...,...,...
718,ZEB1,2
721,ZMYM3,2
724,ZNF429,2
725,ZNF479,2


In [53]:
# Bushman group allOnco

bushman_list = pd.read_csv('C:/Users/renan/Desktop/Labels/Bushman_group_allOnco.tsv', sep='\t')

bushman_list

,symbol,prevSymbols,synonyms,name,refSeqName
1,NKX2-2,NKX2B,NKX2.2,NK2 homeobox 2,NKX2-2
2,MEN1,NaN,NaN,menin 1,MEN1
3,FYN,NaN,"SYN,SLK,MGC45350","FYN proto-oncogene, Src family tyrosine kinase",FYN
4,PTPRE,NaN,PTPE,"protein tyrosine phosphatase, receptor type E",PTPRE
5,PEAR1,MEGF12,"JEDI,FLJ00193",platelet endothelial aggregation receptor 1,PEAR1
...,...,...,...,...,...
2586,ZNF479,NaN,KR19,zinc finger protein 479,ZNF479
2587,ZNRF3,NaN,"KIAA1133,BK747E2.3,FLJ22057,RNF203",zinc and ring finger 3,ZNRF3
2588,ZRSR2,U2AF1L2,"U2AF1-RS2,URP,ZC3H22","zinc finger CCCH-type, RNA binding motif and s...",ZRSR2
2589,LRMDA,C10ORF11,"CDA017,OCA7",leucine rich melanocyte differentiation associ...,C10ORF11


In [54]:
bushman_list2 = bushman_list[['symbol']]

bushman_list2

,symbol
1,NKX2-2
2,MEN1
3,FYN
4,PTPRE
5,PEAR1
...,...
2586,ZNF479
2587,ZNRF3
2588,ZRSR2
2589,LRMDA


In [22]:
# OncoKB

oncokb_list = pd.read_csv('C:/Users/renan/Desktop/Labels/OncoKB_Cancer_Gene_List.tsv', sep='\t')

oncokb_list

,Hugo Symbol,Entrez Gene ID,GRCh37 Isoform,GRCh37 RefSeq,GRCh38 Isoform,GRCh38 RefSeq,# of occurrence within resources (Column D-J),OncoKB Annotated,Is Oncogene,Is Tumor Suppressor Gene,MSK-IMPACT,MSK-HEME,FOUNDATION ONE,FOUNDATION ONE HEME,Vogelstein,SANGER CGC(05/30/2017),Gene Aliases
0,ABL1,25,ENST00000318560,NM_005157.4,ENST00000318560,NM_005157.4,7,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,"ABL, JTK7, c-ABL"
1,AKT1,207,ENST00000349310,NM_001014431.1,ENST00000349310,NM_001014431.1,7,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,"AKT, PKB, PRKBA, RAC, RAC-alpha"
2,ALK,238,ENST00000389048,NM_004304.4,ENST00000389048,NM_004304.4,7,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,CD246
3,AMER1,139285,ENST00000330258,NM_152424.3,ENST00000374869,NM_152424.3,7,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"FAM123B, FLJ39827, RP11-403E24.2, WTX"
4,APC,324,ENST00000257430,NM_000038.5,ENST00000257430,NM_000038.5,7,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"DP2.5, PPP1R46"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,ZFP36L1,677,ENST00000336440,NM_001244698.1,ENST00000336440,NM_001244698.1,1,Yes,No,Yes,No,No,No,No,No,No,"Berg36, RNF162B, TIS11B, cMG1"
1062,ZMYM3,9203,NaN,NaN,NaN,NaN,1,No,No,No,No,No,No,Yes,No,No,"DXS6673E, KIAA0385, ZNF198L2, ZNF261"
1063,ZNF24,7572,NaN,NaN,NaN,NaN,1,No,No,No,No,No,No,Yes,No,No,"KOX17, ZNF191, ZSCAN3, Zfp191"
1064,ZNF331,55422,NaN,NaN,NaN,NaN,1,No,No,No,No,No,No,No,No,Yes,"RITA, ZNF361, ZNF463"


In [23]:
oncokb_list2 = oncokb_list[['Hugo Symbol','Is Oncogene', 'Is Tumor Suppressor Gene']]

oncokb_list2

,Hugo Symbol,Is Oncogene,Is Tumor Suppressor Gene
0,ABL1,Yes,No
1,AKT1,Yes,No
2,ALK,Yes,No
3,AMER1,No,Yes
4,APC,No,Yes
...,...,...,...
1061,ZFP36L1,No,Yes
1062,ZMYM3,No,No
1063,ZNF24,No,No
1064,ZNF331,No,No


In [24]:
oncokb_list2["labels"] = oncokb_list2["Is Oncogene"] + oncokb_list2["Is Tumor Suppressor Gene"]


oncokb_list2

<ipython-input-24-20ffec3aaa93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oncokb_list2["labels"] = oncokb_list2["Is Oncogene"] + oncokb_list2["Is Tumor Suppressor Gene"]


,Hugo Symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
0,ABL1,Yes,No,YesNo
1,AKT1,Yes,No,YesNo
2,ALK,Yes,No,YesNo
3,AMER1,No,Yes,NoYes
4,APC,No,Yes,NoYes
...,...,...,...,...
1061,ZFP36L1,No,Yes,NoYes
1062,ZMYM3,No,No,NoNo
1063,ZNF24,No,No,NoNo
1064,ZNF331,No,No,NoNo


In [27]:
# Pegar a lista NoNo para ser candidatos e alista restante para ser canonico

oncokb_canon = oncokb_list2[oncokb_list2['labels'] != 'NoNo']

oncokb_canon

,Hugo Symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
0,ABL1,Yes,No,YesNo
1,AKT1,Yes,No,YesNo
2,ALK,Yes,No,YesNo
3,AMER1,No,Yes,NoYes
4,APC,No,Yes,NoYes
...,...,...,...,...
1052,WWP1,Yes,No,YesNo
1057,YY1,Yes,No,YesNo
1059,ZBTB20,Yes,Yes,YesYes
1061,ZFP36L1,No,Yes,NoYes


In [28]:
oncokb_candidato = oncokb_list2[oncokb_list2['labels'] == 'NoNo']

oncokb_candidato

,Hugo Symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
127,CSF1R,No,No,NoNo
143,GATA1,No,No,NoNo
146,H3C2,No,No,NoNo
147,IKZF1,No,No,NoNo
168,NSD3,No,No,NoNo
...,...,...,...,...
1058,YY1AP1,No,No,NoNo
1060,ZBTB7A,No,No,NoNo
1062,ZMYM3,No,No,NoNo
1063,ZNF24,No,No,NoNo


In [25]:
oncokb_list2['labels'].value_counts()

NoNo      500
NoYes     267
YesNo     260
YesYes     39
Name: labels, dtype: int64

In [56]:
# Lista final de candidatos juntando todas as fontes de dados e removendo quando tiver algum gene pertencente aos canonicos

# lista_NCG_candidatos5 = parte1 = 3177
# lista_cgc_candidatos3 = parte2 = 151
# bushman_list2 = parte3 = 2580
# oncokb_candidato = parte4 500

# renomear as colunas com os ids para concatenar

oncokb_candidato


,Hugo Symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
127,CSF1R,No,No,NoNo
143,GATA1,No,No,NoNo
146,H3C2,No,No,NoNo
147,IKZF1,No,No,NoNo
168,NSD3,No,No,NoNo
...,...,...,...,...
1058,YY1AP1,No,No,NoNo
1060,ZBTB7A,No,No,NoNo
1062,ZMYM3,No,No,NoNo
1063,ZNF24,No,No,NoNo


In [57]:
parte4 = oncokb_candidato.rename(columns={"Hugo Symbol": "symbol"})

parte4

,symbol,Is Oncogene,Is Tumor Suppressor Gene,labels
127,CSF1R,No,No,NoNo
143,GATA1,No,No,NoNo
146,H3C2,No,No,NoNo
147,IKZF1,No,No,NoNo
168,NSD3,No,No,NoNo
...,...,...,...,...
1058,YY1AP1,No,No,NoNo
1060,ZBTB7A,No,No,NoNo
1062,ZMYM3,No,No,NoNo
1063,ZNF24,No,No,NoNo


In [51]:
parte2 = lista_cgc_candidatos3.rename(columns={"Gene Symbol": "symbol"})

parte2

,symbol,Tier
0,A1CF,2
6,ACSL6,2
13,AKAP9,2
16,AKT3,2
17,ALDH2,2
...,...,...
718,ZEB1,2
721,ZMYM3,2
724,ZNF429,2
725,ZNF479,2


In [58]:
candidatos_list = pd.concat([lista_NCG_candidatos5, parte2, bushman_list2, parte4])

candidatos_list

,symbol,Tier,Is Oncogene,Is Tumor Suppressor Gene,labels
0,ABCF1,NaN,NaN,NaN,NaN
1,ABL1,NaN,NaN,NaN,NaN
6,ACO1,NaN,NaN,NaN,NaN
7,ACTB,NaN,NaN,NaN,NaN
12,ACTG1,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1058,YY1AP1,NaN,No,No,NoNo
1060,ZBTB7A,NaN,No,No,NoNo
1062,ZMYM3,NaN,No,No,NoNo
1063,ZNF24,NaN,No,No,NoNo


In [59]:
candidatos_list2 = candidatos_list.drop_duplicates(subset = ['symbol'], keep = 'first')

candidatos_list2

,symbol,Tier,Is Oncogene,Is Tumor Suppressor Gene,labels
0,ABCF1,NaN,NaN,NaN,NaN
1,ABL1,NaN,NaN,NaN,NaN
6,ACO1,NaN,NaN,NaN,NaN
7,ACTB,NaN,NaN,NaN,NaN
12,ACTG1,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1035,TLE3,NaN,No,No,NoNo
1039,TMSB4XP8,NaN,No,No,NoNo
1046,TUSC3,NaN,No,No,NoNo
1058,YY1AP1,NaN,No,No,NoNo


In [60]:
candidatos_list3 = candidatos_list2[['symbol']]

candidatos_list3

,symbol
0,ABCF1
1,ABL1
6,ACO1
7,ACTB
12,ACTG1
...,...
1035,TLE3
1039,TMSB4XP8
1046,TUSC3
1058,YY1AP1


In [66]:
# Verificando se na lista de candidatos tem algum canonico, e removendo

candidatos_list3['label1'] = 'F'

candidatos_list3

<ipython-input-66-7e782cde6507>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidatos_list3['label1'] = 'F'


,symbol,label,label1
0,ABCF1,F,F
1,ABL1,F,F
6,ACO1,F,F
7,ACTB,F,F
12,ACTG1,F,F
...,...,...,...
1035,TLE3,F,F
1039,TMSB4XP8,F,F
1046,TUSC3,F,F
1058,YY1AP1,F,F


In [67]:
canonicos_list3['label2'] = 'V'

canonicos_list3

<ipython-input-67-4f2cb089ef67>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonicos_list3['label2'] = 'V'


,symbol,label,label2
2,ABL1,V,V
5,ABL2,V,V
17,ACVR1,V,V
32,ACVR1B,V,V
41,ACVR2A,V,V
...,...,...,...
1052,WWP1,V,V
1057,YY1,V,V
1059,ZBTB20,V,V
1061,ZFP36L1,V,V


In [68]:
teste1 = pd.concat([candidatos_list3, canonicos_list3])

teste1

,symbol,label,label1,label2
0,ABCF1,F,F,NaN
1,ABL1,F,F,NaN
6,ACO1,F,F,NaN
7,ACTB,F,F,NaN
12,ACTG1,F,F,NaN
...,...,...,...,...
1052,WWP1,V,NaN,V
1057,YY1,V,NaN,V
1059,ZBTB20,V,NaN,V
1061,ZFP36L1,V,NaN,V


In [74]:
teste2 = teste1.replace(np.NaN , '-')

In [75]:
teste2["labels"] = teste2["label1"] + teste2["label2"]


teste2

,symbol,label,label1,label2,labels
0,ABCF1,F,F,-,F-
1,ABL1,F,F,-,F-
6,ACO1,F,F,-,F-
7,ACTB,F,F,-,F-
12,ACTG1,F,F,-,F-
...,...,...,...,...,...
1052,WWP1,V,-,V,-V
1057,YY1,V,-,V,-V
1059,ZBTB20,V,-,V,-V
1061,ZFP36L1,V,-,V,-V


In [79]:
# removendo as duplicadas e exluido a primeira aparição que é o grupo F (candidatos)
# depois removendo todos as linhas V (canonicos)
# restando só os candidatos que não apareceram nos canonicos

teste3 = teste2.drop_duplicates(subset = ['symbol'], keep = 'last')

teste3

,symbol,label,label1,label2,labels
0,ABCF1,F,F,-,F-
6,ACO1,F,F,-,F-
7,ACTB,F,F,-,F-
53,ACVR2B,F,F,-,F-
54,ADAM10,F,F,-,F-
...,...,...,...,...,...
1052,WWP1,V,-,V,-V
1057,YY1,V,-,V,-V
1059,ZBTB20,V,-,V,-V
1061,ZFP36L1,V,-,V,-V


In [81]:
teste3['label'].value_counts()

F    4135
V     917
Name: label, dtype: int64

In [83]:
teste4 = teste3[teste3['label'] == 'F']

teste4

,symbol,label,label1,label2,labels
0,ABCF1,F,F,-,F-
6,ACO1,F,F,-,F-
7,ACTB,F,F,-,F-
53,ACVR2B,F,F,-,F-
54,ADAM10,F,F,-,F-
...,...,...,...,...,...
1035,TLE3,F,F,-,F-
1039,TMSB4XP8,F,F,-,F-
1046,TUSC3,F,F,-,F-
1058,YY1AP1,F,F,-,F-


In [84]:
teste5 = teste4[['symbol']]

teste5

,symbol
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
1035,TLE3
1039,TMSB4XP8
1046,TUSC3
1058,YY1AP1


### Associados

In [67]:
omim = pd.read_csv('C:/Users/renan/Desktop/Labels/mim2gene.txt', sep='\t')

omim

,MIM Number,MIM Entry Type,Entrez Gene ID (NCBI),Approved Gene Symbol (HGNC),Ensembl Gene ID (Ensembl)
0,100050,predominantly phenotypes,NaN,NaN,NaN
1,100070,phenotype,100329167.0,NaN,NaN
2,100100,phenotype,NaN,NaN,NaN
3,100200,predominantly phenotypes,NaN,NaN,NaN
4,100300,phenotype,NaN,NaN,NaN
...,...,...,...,...,...
27569,619698,phenotype,NaN,NaN,NaN
27570,619700,gene,148345.0,C1orf127,ENSG00000175262
27571,619703,gene,NaN,NaN,NaN
27572,619704,gene,55854.0,ZC3H15,ENSG00000065548


In [86]:
omim['MIM Entry Type '].value_counts()

gene                        16652
phenotype                    7800
predominantly phenotypes     1758
moved/removed                1337
gene/phenotype                 27
Name: MIM Entry Type , dtype: int64

In [92]:
cpdb_list = pd.read_csv('C:/Users/renan/Desktop/Labels/CPDB_pathways_genes.tab', sep= '\t')

cpdb_list

,pathway,external_id,source,hgnc_symbol_ids
0,"Alanine, aspartate and glutamate metabolism - ...",path:hsa00250,KEGG,"NIT2,ASNS,GLUL,ABAT,ADSL,GLS,ASS1,GAD1,GAD2,CA..."
1,Folate biosynthesis - Homo sapiens (human),path:hsa00790,KEGG,"PCBD2,MOCOS,PCBD1,MOCS2,MOCS1,DHFR2,PAH,AKR1C3..."
2,Complement and coagulation cascades - Homo sap...,path:hsa04610,KEGG,"F8,KNG1,F11,CD55,C1QC,VSIG4,CD59,PROS1,C1QB,C8..."
3,Citrate cycle (TCA cycle) - Homo sapiens (human),path:hsa00020,KEGG,"PCK2,PCK1,CS,MDH2,MDH1,FH,IDH3A,OGDH,IDH3B,IDH..."
4,Renin-angiotensin system - Homo sapiens (human),path:hsa04614,KEGG,"ACE,REN,AGT,ENPEP,CPA3,ATP6AP2,NLN,LNPEP,CTSA,..."
...,...,...,...,...
4676,Intracellular oxygen transport,R-HSA-8981607,Reactome,"CYGB,NGB,MB"
4677,Translesion synthesis by REV1,R-HSA-110312,Reactome,"MAD2L2,PCNA,RFC1,RFC5,RFC4,RPS27A,RPA1,RPA3,RP..."
4678,Synthesis of glycosylphosphatidylinositol (GPI),R-HSA-162710,Reactome,"PIGA,PIGB,PIGC,PIGF,PIGG,PIGH,PIGL,PIGM,PIGN,P..."
4679,Synaptic adhesion-like molecules,R-HSA-8849932,Reactome,"DLG1,GRIN2B,DLG3,GRIN2C,PTPRS,GRIN2D,LRFN2,GRI..."


In [96]:
cpdb_list_cancer = cpdb_list.loc[cpdb_list['pathway'].str.contains("cancer", case=False)]

cpdb_list_cancer

,pathway,external_id,source,hgnc_symbol_ids
11,Non-small cell lung cancer - Homo sapiens (human),path:hsa05223,KEGG,"FHIT,MAP2K1,RASSF1,PIK3CD,CDKN1A,RASSF5,BAX,FO..."
52,PD-L1 expression and PD-1 checkpoint pathway i...,path:hsa05235,KEGG,"PPP3R2,PPP3R1,IKBKB,AKT1,AKT2,AKT3,IKBKG,EGF,M..."
55,Central carbon metabolism in cancer - Homo sap...,path:hsa05230,KEGG,"PDGFRB,PDGFRA,MAP2K1,PIK3CD,HRAS,SLC7A5,EGFR,H..."
56,Choline metabolism in cancer - Homo sapiens (h...,path:hsa05231,KEGG,"LYPLA1,AKT1,AKT2,AKT3,EGF,PIK3CA,PIK3CB,PIK3CD..."
72,Bladder cancer - Homo sapiens (human),path:hsa05219,KEGG,"RASSF1,CDKN1A,EGFR,MAP2K2,MAP2K1,ARAF,RAF1,EGF..."
...,...,...,...,...
4016,stathmin and breast cancer resistance to antim...,stathminpathway,BioCarta,"TUBA8,STMN1,PRKAR2A,PRKAR2B,CD2,MAPK13,TUBA3C,..."
4426,Constitutive Signaling by Ligand-Responsive EG...,R-HSA-1236382,Reactome,"SHC1,SOS1,PIK3CA,HRAS,NRAS,EGFR,CBL,GAB1,PLCG1..."
4427,Signaling by Ligand-Responsive EGFR Variants i...,R-HSA-5637815,Reactome,"SHC1,SOS1,PIK3CA,HRAS,NRAS,EGFR,CBL,GAB1,PLCG1..."
4585,Signaling by EGFR in Cancer,R-HSA-1643713,Reactome,"TGFA,AREG,EPGN,SHC1,SOS1,HBEGF,PIK3CA,HRAS,NRA..."


In [115]:
cpdb_list_cancer2 = cpdb_list_cancer[['hgnc_symbol_ids']]

cpdb_list_cancer2

,hgnc_symbol_ids
11,"FHIT,MAP2K1,RASSF1,PIK3CD,CDKN1A,RASSF5,BAX,FO..."
52,"PPP3R2,PPP3R1,IKBKB,AKT1,AKT2,AKT3,IKBKG,EGF,M..."
55,"PDGFRB,PDGFRA,MAP2K1,PIK3CD,HRAS,SLC7A5,EGFR,H..."
56,"LYPLA1,AKT1,AKT2,AKT3,EGF,PIK3CA,PIK3CB,PIK3CD..."
72,"RASSF1,CDKN1A,EGFR,MAP2K2,MAP2K1,ARAF,RAF1,EGF..."
...,...
4016,"TUBA8,STMN1,PRKAR2A,PRKAR2B,CD2,MAPK13,TUBA3C,..."
4426,"SHC1,SOS1,PIK3CA,HRAS,NRAS,EGFR,CBL,GAB1,PLCG1..."
4427,"SHC1,SOS1,PIK3CA,HRAS,NRAS,EGFR,CBL,GAB1,PLCG1..."
4585,"TGFA,AREG,EPGN,SHC1,SOS1,HBEGF,PIK3CA,HRAS,NRA..."


In [116]:
cpdb_list_cancer2['hgnc_symbol_ids'] = cpdb_list_cancer2['hgnc_symbol_ids'].str.split(',')
df = cpdb_list_cancer2.explode('hgnc_symbol_ids')

<ipython-input-116-992077dce5bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpdb_list_cancer2['hgnc_symbol_ids'] = cpdb_list_cancer2['hgnc_symbol_ids'].str.split(',')


In [117]:
df

,hgnc_symbol_ids
11,FHIT
11,MAP2K1
11,RASSF1
11,PIK3CD
11,CDKN1A
...,...
4650,KRAS
4650,SOS1
4650,GRB2
4650,FGF23


In [118]:
df.to_csv('C:/Users/renan/Desktop/Labels/lista_teste.csv')

In [120]:
lista_cpdb_teste = pd.read_csv('C:/Users/renan/Desktop/Labels/lista_teste.csv')

lista_cpdb_teste2 = lista_cpdb_teste.drop(columns=['Unnamed: 0'])

lista_cpdb_teste2

,hgnc_symbol_ids
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
4328,KRAS
4329,SOS1
4330,GRB2
4331,FGF23


In [91]:
# Os genes negativos na verdade são todos que restaram de todas as listas criadas acima, a lista de canonicos, a lista de candidatos e a lista de associados a doenças
# que deve ficar de fora dos que são considerados negativos 

# Logo temos duas listas de genes associados a doenças, a primeira é do OMIM

omim = pd.read_csv('C:/Users/renan/Desktop/Labels/All_Diseases_OMIM.txt', sep=' ')

omim

,ENTREZID,SYMBOL
1,374654,KIF7
2,1302,COL11A2
3,8626,TP63
4,2657,GDF1
5,5190,PEX6
...,...,...
2142,201456,FBXO15
2143,153020,RASGEF1B
2144,441525,SPANXN4
2145,100127911,ST13P12


In [97]:
omim2 = omim[['SYMBOL']]

omim2

,SYMBOL
1,KIF7
2,COL11A2
3,TP63
4,GDF1
5,PEX6
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [121]:
# Lista completa de genes associados a doenças

# Parte1 -- lista_cpdb_teste2 = 4333
# Parte2 -- omim2 = 2143

parte1_doencas = lista_cpdb_teste2.rename(columns={"hgnc_symbol_ids": "symbol"})

parte1_doencas

,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
4328,KRAS
4329,SOS1
4330,GRB2
4331,FGF23


In [122]:
parte2_doencas = omim2.rename(columns={"SYMBOL": "symbol"})

parte2_doencas

,symbol
1,KIF7
2,COL11A2
3,TP63
4,GDF1
5,PEX6
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [123]:
associados_list = pd.concat([parte1_doencas, parte2_doencas])

associados_list

,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [124]:
associados_list2 = associados_list.drop_duplicates(subset = ['symbol'], keep = 'first')

associados_list2

,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


### Organizando as listas completas e traduzindo

In [125]:
# Genes candidatos 

teste5

,symbol
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
1035,TLE3
1039,TMSB4XP8
1046,TUSC3
1058,YY1AP1


In [133]:
teste5.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/candidatos.csv', sep='\t')

In [126]:
# Genes canonicos de cancer

canonicos_list4 = canonicos_list3[['symbol']]

canonicos_list4

,symbol
2,ABL1
5,ABL2
17,ACVR1
32,ACVR1B
41,ACVR2A
...,...
1052,WWP1
1057,YY1
1059,ZBTB20
1061,ZFP36L1


In [131]:
canonicos_list4.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/canonicos.csv', sep='\t')

In [127]:
# Lista de genes associados a doenças que devem estar fora dos negativos

associados_list2

,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [132]:
associados_list2.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/associados.csv', sep='\t')

In [290]:
# Lista de genes que tenho disponiveis com features associadas

label_unity = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_labels.tsv', sep= '\t')

label_unity2 = label_unity[['gene']]

label_unity2

,gene
0,A1BG
1,A1CF
2,A2M
3,A2ML1
4,A3GALT2
...,...
19597,ZYG11A
19598,ZYG11B
19599,ZYX
19600,ZZEF1


In [134]:
# Leitura dos dicionários dos genes para tradução
# Vou precisar juntar todas as listas para abranger o maior numero de genes possiveis

MULTINET_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_MULTINET_final2.tsv', sep='\t', index_col='Unnamed: 0')

CPDB_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_CPDB_final2.tsv', sep='\t', index_col='Unnamed: 0')

IREF_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_IREF_final2.tsv', sep='\t', index_col='Unnamed: 0')

PCNET_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_PCNET_final2.tsv', sep='\t', index_col='Unnamed: 0')

STRINGdb_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_STRINGdb_final2.tsv', sep='\t', index_col='Unnamed: 0')

IREF_2015_genes = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/traducao_genes/lista_IREF_2015_final2.tsv', sep='\t', index_col='Unnamed: 0')

In [135]:
all_dicts = pd.concat([MULTINET_genes, CPDB_genes, IREF_genes, PCNET_genes, STRINGdb_genes, IREF_2015_genes])

all_dicts

,Input,Match type,Approved symbol,HGNC ID
0,A1CF,Approved symbol,A1CF,HGNC:24086
1,A2BP1,Alias symbol,RBFOX1,HGNC:18222
2,A2M,Approved symbol,A2M,HGNC:7
3,AAAS,Approved symbol,AAAS,HGNC:13666
4,AACS,Approved symbol,AACS,HGNC:21298
...,...,...,...,...
12650,ZXDC,Approved symbol,ZXDC,HGNC:28160
12651,ZYG11B,Approved symbol,ZYG11B,HGNC:25820
12652,ZYX,Approved symbol,ZYX,HGNC:13200
12653,ZZEF1,Approved symbol,ZZEF1,HGNC:29027


In [136]:
all_dicts2 = all_dicts.drop_duplicates(subset = ['Input'], keep = 'first')

all_dicts2

,Input,Match type,Approved symbol,HGNC ID
0,A1CF,Approved symbol,A1CF,HGNC:24086
1,A2BP1,Alias symbol,RBFOX1,HGNC:18222
2,A2M,Approved symbol,A2M,HGNC:7
3,AAAS,Approved symbol,AAAS,HGNC:13666
4,AACS,Approved symbol,AACS,HGNC:21298
...,...,...,...,...
12133,WDR16,Previous symbol,CFAP52,HGNC:16053
12176,WDR96,Previous symbol,CFAP43,HGNC:26684
12234,XAGE2B,Previous symbol,XAGE2,HGNC:4112
12582,ZNF664-FAM101A,Unmatched,NaN,NaN


In [137]:
# Substituir a coluna gene da lista pela Approved symbol do dicionário

canonicos_list4.replace(all_dicts2.set_index('Input').to_dict()['Approved symbol'], inplace = True)

canonicos_list4

C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:4378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,symbol
2,ABL1
5,ABL2
17,ACVR1
32,ACVR1B
41,ACVR2A
...,...
1052,WWP1
1057,YY1
1059,ZBTB20
1061,ZFP36L1


In [139]:
# Remove as traduções que não tinham resultado (Unmatched) e remove as duplicadas de casos que diferentes genes traduziram para um mesmo

canonicos = canonicos_list4[canonicos_list4['symbol'].notna()]

canonicos2 = canonicos.drop_duplicates(subset = ['symbol'], keep = 'first')

canonicos2

,symbol
2,ABL1
5,ABL2
17,ACVR1
32,ACVR1B
41,ACVR2A
...,...
1052,WWP1
1057,YY1
1059,ZBTB20
1061,ZFP36L1


In [146]:
canon1 = canonicos2.rename(columns={"symbol": "gene"})

canon1.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/canonicos_final.tsv', sep='\t')

In [140]:
# Substituir a coluna gene da lista pela Approved symbol do dicionário

teste5.replace(all_dicts2.set_index('Input').to_dict()['Approved symbol'], inplace = True)

teste5

C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:4378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,symbol
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
1035,TLE3
1039,TMSB4XP8
1046,TUSC3
1058,YY1AP1


In [141]:
# Remove as traduções que não tinham resultado (Unmatched) e remove as duplicadas de casos que diferentes genes traduziram para um mesmo

candidatos = teste5[teste5['symbol'].notna()]

candidatos2 = candidatos.drop_duplicates(subset = ['symbol'], keep = 'first')

candidatos2

,symbol
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
1035,TLE3
1039,TMSB4XP8
1046,TUSC3
1058,YY1AP1


In [147]:
cand1 = candidatos2.rename(columns={"symbol": "gene"})

cand1.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/candidatos_final.tsv', sep='\t')

In [142]:
# Substituir a coluna gene da lista pela Approved symbol do dicionário

associados_list2.replace(all_dicts2.set_index('Input').to_dict()['Approved symbol'], inplace = True)

associados_list2

C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:4378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [143]:
# Remove as traduções que não tinham resultado (Unmatched) e remove as duplicadas de casos que diferentes genes traduziram para um mesmo

associados = associados_list2[associados_list2['symbol'].notna()]

associados2 = associados.drop_duplicates(subset = ['symbol'], keep = 'first')

associados2

,symbol
0,FHIT
1,MAP2K1
2,RASSF1
3,PIK3CD
4,CDKN1A
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [148]:
asso1 = associados2.rename(columns={"symbol": "gene"})

asso1.to_csv('C:/Users/renan/Desktop/Labels/listas_finais/associados_final.tsv', sep='\t')

### Definindo labels

In [291]:
# Leitura dos arquivos finais traduzidos e montagem das colunas de labels

genes_canonicos = pd.read_csv('C:/Users/renan/Desktop/Labels/listas_finais/canonicos_final.tsv', sep='\t', index_col='Unnamed: 0')
genes_candidatos = pd.read_csv('C:/Users/renan/Desktop/Labels/listas_finais/candidatos_final.tsv', sep='\t', index_col='Unnamed: 0')
genes_associados = pd.read_csv('C:/Users/renan/Desktop/Labels/listas_finais/associados_final.tsv', sep='\t', index_col='Unnamed: 0')

In [265]:
genes_candidatos['label'] = '**'
genes_associados['label'] = '--'

In [292]:
label_unity2

,gene
0,A1BG
1,A1CF
2,A2M
3,A2ML1
4,A3GALT2
...,...
19597,ZYG11A
19598,ZYG11B
19599,ZYX
19600,ZZEF1


In [293]:
label_unity2['gene'].isin(genes_canonicos['gene']).value_counts()

False    18695
True       907
Name: gene, dtype: int64

In [294]:
#label_unity2['label'] = np.nan

label_unity2['label'] = label_unity2['gene'].isin(genes_canonicos['gene'])

label_unity2

,gene,label
0,A1BG,False
1,A1CF,True
2,A2M,False
3,A2ML1,False
4,A3GALT2,False
...,...,...
19597,ZYG11A,False
19598,ZYG11B,False
19599,ZYX,False
19600,ZZEF1,False


In [295]:
label_unity2['label'].value_counts()

False    18695
True       907
Name: label, dtype: int64

In [296]:
# criando uma lista com candidatos e associados a doenças que serão unlabel

lista1 = pd.concat([genes_candidatos, genes_associados])

lista1

,gene
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [297]:
# Nessa remoção de genes duplicados entre as duas listas de candidatos e genes associados eu optei por manter as duplicadas do genes candidatos

lista2 = lista1.drop_duplicates(subset = ['gene'], keep = 'first')

lista2

,gene
0,ABCF1
6,ACO1
7,ACTB
53,ACVR2B
54,ADAM10
...,...
2142,FBXO15
2143,RASGEF1B
2144,SPANXN4
2145,ST13P12


In [298]:
lista2['gene'].isin(genes_canonicos['gene']).value_counts()

False    6032
True      553
Name: gene, dtype: int64

In [299]:
lista2['label'] = lista2['gene'].isin(genes_canonicos['gene'])

lista2

<ipython-input-299-772d70b236bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lista2['label'] = lista2['gene'].isin(genes_canonicos['gene'])


,gene,label
0,ABCF1,False
6,ACO1,False
7,ACTB,False
53,ACVR2B,False
54,ADAM10,False
...,...,...
2142,FBXO15,False
2143,RASGEF1B,False
2144,SPANXN4,False
2145,ST13P12,False


In [300]:
lista2['label'].value_counts()

False    6032
True      553
Name: label, dtype: int64

In [301]:
lista3 = lista2[lista2['label'] == False]

lista3

,gene,label
0,ABCF1,False
6,ACO1,False
7,ACTB,False
53,ACVR2B,False
54,ADAM10,False
...,...,...
2142,FBXO15,False
2143,RASGEF1B,False
2144,SPANXN4,False
2145,ST13P12,False


In [302]:
# Comparando os 6032 genes com os 19602 da rede

label_unity2['gene'].isin(lista3['gene']).value_counts()

False    13825
True      5777
Name: gene, dtype: int64

In [303]:
label_unity2['unlabel'] = label_unity2['gene'].isin(lista3['gene'])

label_unity2

,gene,label,unlabel
0,A1BG,False,False
1,A1CF,True,False
2,A2M,False,False
3,A2ML1,False,True
4,A3GALT2,False,False
...,...,...,...
19597,ZYG11A,False,False
19598,ZYG11B,False,False
19599,ZYX,False,False
19600,ZZEF1,False,True


In [305]:
label_unity2['label'].value_counts()

# A conta fecha com 907 genes canonicos dentro da minha rede (na culuna label)
# E 5777 genes candidatos e associados que serão considerados unlabel (na culuna unlabel)

False    18695
True       907
Name: label, dtype: int64

In [306]:
# A partir das duas culunas posso unifica-las e fazer mais uma verificação se não tem nenhum true/true (caso de um canonico se repetir)
# Os genes restantes serão negativos

# Unificando as duas colunas a legenda será a segunte:
    # TrueFalse = gene canonico
    # TrueTrue = Não pode acontecer gene que é canonico não deve estar na lista de candidatos e associados
    # FalseTrue = gene candidato ou associado, essa categoria deve ser modificada para unlabel (NaN)
    # FalseFalse = genes negativos

# Primeiro preciso transformar em caracter pois recenhece como operação logica

booleanDictionary = {True: 'TRUE', False: 'FALSE'}
label_unity2 = label_unity2.replace(booleanDictionary)

label_unity2["labels_uni"] = label_unity2["label"] + label_unity2["unlabel"]

label_unity2

,gene,label,unlabel,labels_uni
0,A1BG,FALSE,FALSE,FALSEFALSE
1,A1CF,TRUE,FALSE,TRUEFALSE
2,A2M,FALSE,FALSE,FALSEFALSE
3,A2ML1,FALSE,TRUE,FALSETRUE
4,A3GALT2,FALSE,FALSE,FALSEFALSE
...,...,...,...,...
19597,ZYG11A,FALSE,FALSE,FALSEFALSE
19598,ZYG11B,FALSE,FALSE,FALSEFALSE
19599,ZYX,FALSE,FALSE,FALSEFALSE
19600,ZZEF1,FALSE,TRUE,FALSETRUE


In [307]:
label_unity2['labels_uni'].value_counts()

FALSEFALSE    12918
FALSETRUE      5777
TRUEFALSE       907
Name: labels_uni, dtype: int64

In [310]:
label_unity3 = label_unity2.replace({'TRUEFALSE': True, 'FALSETRUE': np.nan, 'FALSEFALSE': False})

In [313]:
label_unity4 = label_unity3.drop(columns=['label', 'unlabel'])

label_unity5 = label_unity4.rename(columns={"labels_uni": "label"})

label_unity5

,gene,label
0,A1BG,False
1,A1CF,True
2,A2M,False
3,A2ML1,NaN
4,A3GALT2,False
...,...,...
19597,ZYG11A,False
19598,ZYG11B,False
19599,ZYX,False
19600,ZZEF1,NaN


In [314]:
label_unity5['label'].value_counts()

False    12918
True       907
Name: label, dtype: int64

In [315]:
label_unity5.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_labels_semisupervised.tsv', sep='\t')